In [1]:
import os
import json
import re
from dotenv import load_dotenv
from openai import AzureOpenAI, OpenAIError

load_dotenv()  # .env 파일에 있는 환경변수 로드

aoi_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoi_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoi_gen_model = os.getenv("AZURE_GENERATION_MODEL")
aoi_version = os.getenv("AZURE_GENERATION_MODEL_VERSION")
client = AzureOpenAI(azure_endpoint=aoi_endpoint,api_key=aoi_api_key, api_version=aoi_version)


In [9]:
### Build Index
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings


# Set embeddings
embedding_model = AzureOpenAIEmbeddings(
                                   deployment="text-embedding-ada-002",
                                   openai_api_key=aoi_api_key,
                                   azure_endpoint=aoi_endpoint,
                                   api_version=aoi_version
                                  )

# Docs to index
urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io"
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={'k': 4}, # number of documents to retrieve
            )

In [83]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )



# LLM with function call
llm = AzureChatOpenAI(
    azure_endpoint=aoi_endpoint,
    api_key=aoi_api_key, 
    api_version=aoi_version,
    deployment_name=aoi_gen_model
)
structured_llm_grader = llm.with_structured_output(
    GradeDocuments,
    method="function_calling"
)
# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
    """
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [74]:
docs_to_use = []
for doc in docs:
    print(doc.page_content, '\n', '-'*50)
    # Invoke the grader with a dictionary input, expecting a structured GradeDocuments output
    response = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    binary_answer = response.binary_score.lower()

    print(binary_answer,'\n')
    if binary_answer == 'yes':
        docs_to_use.append(doc)

say, scalable and highly secure code. By decomposing the overall task into subtasks, we can optimize the subtasks better.Perhaps most important, the multi-agent design pattern gives us, as developers, a framework for breaking down complex tasks into subtasks. When writing code to run on a single CPU, we often break our program up into different processes or threads. This is a useful abstraction that lets us decompose a task, like implementing a web browser, into subtasks that are easier to code. I find thinking through multi-agent roles to be a useful abstraction as well.In many companies, managers routinely decide what roles to hire, and then how to split complex projects — like writing a large piece of software or preparing a research report — into smaller tasks to assign to employees with different specialties. Using multiple agents is analogous. Each agent implements its own workflow, has its own memory (itself a rapidly evolving area in agentic technology: how can an agent remembe

In [65]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# LLM
llm = AzureChatOpenAI(
    azure_endpoint=aoi_endpoint,
    api_key=aoi_api_key, 
    api_version=aoi_version,
    deployment_name=aoi_gen_model
)
# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question})
print(generation)

The different kinds of agentic design patterns mentioned in the retrieved documents are:

1. **Reflection**: Involves the agent reflecting on its own output and providing critical feedback to improve its response with each iteration.
2. **Tool Use**: This pattern was mentioned but not described in the retrieved documents.
3. **Planning**: An agent autonomously plans the sequence of steps to execute in order to accomplish a larger task.
4. **Multi-Agent Collaboration**: Involves using multiple agents, each with distinct roles, to collaboratively handle different subtasks of a complex task.


### 주요 학습 내용: 검증 및 개선 단계를 통한 RAG 파이프라인 강화

이번 스크립트 실습과 논의를 통해, 기본적인 RAG (Retrieval-Augmented Generation) 파이프라인에 **검증(Validation) 및 개선(Refinement) 단계를 추가**하여, 검색된 정보의 **정확성(Accuracy)과 관련성(Relevance)을 보장**하고 전체 프로세스를 강화하는 방법을 배웠습니다. 이 과정에서 Pydantic과 LangChain의 `with_structured_output` 기능을 효과적으로 활용하여 LLM 응답의 신뢰성을 높이는 것이 핵심이었습니다.

**핵심 구현 방식 및 효과:**

* **1. 문제 인식 및 목표 설정:**
    * 단순 RAG는 관련성이 낮거나 부정확한 정보를 검색하여 최종 답변의 품질을 저하시키거나, LLM이 환각(Hallucination)을 일으킬 수 있습니다.
    * **목표:** 검색된 정보의 품질을 확인하고 개선하여 RAG 파이프라인의 신뢰도를 높입니다.

* **2. 검증/개선 단계 구현 (문서 관련성 평가):**
    * **Pydantic 모델 정의 (`GradeDocuments`):**
        * LLM이 수행할 작업(관련성 판단)의 결과(예: 'yes'/'no')를 **명확하고 구조화된 형식**으로 정의했습니다.
        * `Field`의 `description`을 사용하여 각 필드의 의미와 기대값을 LLM에게 **힌트로 제공**했습니다. (단, 시스템 프롬프트의 직접적인 지시와 충돌 시 프롬프트가 우선될 수 있음을 인지했습니다.)
        * 이는 LLM의 응답에서 모호함을 제거하고 **일관된 형식**을 갖도록 유도합니다.
    * **`with_structured_output` 활용:**
        * LLM(`AzureChatOpenAI`)이 **반드시 정의된 Pydantic 구조에 따라 응답하도록 강제**했습니다 (`method="function_calling"` 사용).
        * 이를 통해 LLM의 출력을 프로그램 코드에서 **신뢰성 있게 파싱하고 활용**할 수 있게 되었습니다 (`response.binary_score` 와 같이 직접 접근 가능).
    * **자동 필터링 로직:**
        * 구조화되고 검증된 LLM의 판단 결과('yes'/'no')를 바탕으로, 관련 없는 문서를 **자동으로 필터링**하는 로직을 손쉽게 구현했습니다.

* **3. 파이프라인 강화 효과:**
    * **향상된 관련성 및 정확성:** 최종 답변 생성 LLM에게 **더 관련성 높은 정보만을 컨텍스트로 제공**함으로써, 답변의 정확성과 주제 관련성을 높였습니다.
    * **환각 제어 (간접적):** LLM의 출력을 특정 구조로 제한하고, 검증된 컨텍스트를 제공함으로써 LLM이 자유롭게 정보를 지어내는 **환각 현상을 억제**하는 데 도움을 줄 수 있습니다.
    * **신뢰성 및 예측 가능성 증대:** 전체 RAG 파이프라인의 작동 방식을 더 예측 가능하게 만들고, 최종 결과물에 대한 **신뢰도**를 향상시켰습니다.

결론적으로, Pydantic을 사용한 명확한 데이터 스키마 정의와 `with_structured_output`을 통한 LLM 출력 제어 및 검증은, LLM 기반 애플리케이션에서 **정보의 품질을 관리하고 개선**하여 **더욱 정확하고 신뢰할 수 있는 시스템**을 구축하기 위한 핵심적인 기법임을 배웠습니다. 이는 단순 RAG를 넘어서는 중요한 개선 단계입니다.